In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [34]:
df = pd.read_csv("final_named_data.csv")

corr = df.corr(numeric_only=True)
#plt.figure(figsize=(15, 12))
#sns.heatmap(corr, cmap="coolwarm", annot=False, fmt=".2f", center=0)
#plt.title("Macierz korelacji")
#plt.show()
corr_matrix = df.corr(numeric_only=True).abs()
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        value = corr_matrix.iloc[i, j]
        if value > 0.9:
            col1 = corr_matrix.columns[i]
            col2 = corr_matrix.columns[j]
            print(f"{col1} ⬌ {col2} = {value:.2f}")


id ⬌ Unnamed: 0 = 0.92
cp ⬌ pncaden = 0.91
tpeakbpd ⬌ trestbps = 0.94
restef ⬌ Unnamed: 0 = 0.96
restef ⬌ id = 0.96
restef ⬌ chol = 1.00
restef ⬌ cigs = 1.00
restef ⬌ years = 1.00
restef ⬌ fbs = 0.96
restef ⬌ restecg = 0.96
restef ⬌ ekgyr = 0.94
restef ⬌ dig = 0.94
restef ⬌ diuretic = 0.94
restef ⬌ 27 = 0.94
restef ⬌ thaltime = 1.00
restef ⬌ thalach = 1.00
restef ⬌ thalrest = 1.00
restef ⬌ trestbpd = 0.90
restef ⬌ tpeakbpd = 1.00
restef ⬌ exerckm = 1.00
restwm ⬌ famhist = 0.94
restwm ⬌ thaltime = 1.00
restwm ⬌ exang = 1.00
restwm ⬌ rldv5 = 0.99
restwm ⬌ restef = 1.00
52 ⬌ famhist = 1.00
cyr ⬌ ekgyr = 0.99
cyr ⬌ restef = 0.96
laddist ⬌ restef = 1.00
laddist ⬌ 52 = 1.00
diag ⬌ restef = 0.96
diag ⬌ restwm = 0.94
diag ⬌ 52 = 1.00
cxmain ⬌ 52 = 1.00
ramus ⬌ restef = 0.96
ramus ⬌ restwm = 0.94
om1 ⬌ restef = 0.96
om1 ⬌ restwm = 0.94
om2 ⬌ restef = 0.96
om2 ⬌ restwm = 0.94
rcadist ⬌ restwm = 0.94
lvx3 ⬌ restef = 0.95
lvx3 ⬌ restwm = 0.94
lvf ⬌ restef = 1.00
junk: ⬌ restwm = 1.00
76 ⬌ thaltime

In [35]:
df.drop(columns=["Unnamed: 0", "id"], inplace=True) #to tylko indeksy, można usunąć

#śmiało można też usunąć kolumny z korelacją równą 1.0 - najpewniej są duplikatami
correlated_cols = []
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if corr_matrix.iloc[i, j] == 1.0:
            col1 = corr_matrix.columns[i]
            col2 = corr_matrix.columns[j]
            correlated_cols.append((col1, col2))

correlated_df = pd.DataFrame(correlated_cols, columns = ["Kolumna 1", "Kolumna 2"])
print(correlated_df)

   Kolumna 1 Kolumna 2
0     restef      chol
1     restef      cigs
2     restef     years
3     restef  thaltime
4     restef   exerckm
5     restwm  thaltime
6     restwm     exang
7     restwm    restef
8         52   famhist
9    laddist        52
10      diag        52
11    cxmain        52
12       lvf    restef
13        81   restckm
14        81   exerckm
15        84  painexer
16        84   relrest
17        84     smoke
18        84   famhist
19        84  diuretic
20        84       cmo


In [36]:
#lista kolumn do usunięcia (jedna z każdej pary)
to_drop = set()

for col1, col2 in correlated_cols:
    if col2 not in to_drop:
        to_drop.add(col2)

print(to_drop)


{'diuretic', 'smoke', 'painexer', 'restckm', 'thaltime', 'cmo', 'exang', 'restef', 'cigs', '52', 'chol', 'years', 'exerckm', 'famhist', 'relrest'}


In [37]:
#usuwanie takowych kolumn
df.drop(columns=list(to_drop), inplace=True)

print("Usunięto", to_drop)

Usunięto {'diuretic', 'smoke', 'painexer', 'restckm', 'thaltime', 'cmo', 'exang', 'restef', 'cigs', '52', 'chol', 'years', 'exerckm', 'famhist', 'relrest'}


In [38]:
#NaN
missing = df.isna().sum().sort_values(ascending=False)
missing = missing[missing > 0]
print(missing)

restwm     1535
77         1504
76         1499
51         1479
84         1454
           ... 
htn          44
lvf          39
ccf           6
restecg       2
cp            1
Length: 62, dtype: int64


In [39]:
def content_analysis(df, column):
    dane = df[column].dropna()
    
    print(f"Statystyki kolumny: {column}")
    print(f"Min: {dane.min()}")
    print(f"Maks: {dane.max()}")
    print(f"Średnia: {dane.mean():.2f}")
    print(f"Mediana: {dane.median():.2f}")
    print(f"Liczba braków: {df[column].isna().sum()}\n")

In [40]:
#wyliczenie min, max oraz mean dla wszystkich kolumn z brakami
columns_with_nan = df.columns[df.isna().sum() > 0]

for col in columns_with_nan:
    print("="*40)
    content_analysis(df, col)

Statystyki kolumny: ccf
Min: 0.0
Maks: 17541960.0
Średnia: 3233929.29
Mediana: 7620.00
Liczba braków: 6

Statystyki kolumny: painloc
Min: 0.0
Maks: 1.0
Średnia: 0.90
Mediana: 1.00
Liczba braków: 446

Statystyki kolumny: pncaden
Min: 0.0
Maks: 3.0
Średnia: 2.12
Mediana: 3.00
Liczba braków: 789

Statystyki kolumny: cp
Min: 1.0
Maks: 4.0
Średnia: 3.16
Mediana: 3.00
Liczba braków: 1

Statystyki kolumny: trestbps
Min: 80.0
Maks: 200.0
Średnia: 131.27
Mediana: 130.00
Liczba braków: 58

Statystyki kolumny: htn
Min: 0.0
Maks: 1.0
Średnia: 0.48
Mediana: 0.00
Liczba braków: 44

Statystyki kolumny: fbs
Min: 0.0
Maks: 1.0
Średnia: 0.21
Mediana: 0.00
Liczba braków: 92

Statystyki kolumny: restecg
Min: 0.0
Maks: 2.0
Średnia: 0.62
Mediana: 0.00
Liczba braków: 2

Statystyki kolumny: ekgmo
Min: 1.0
Maks: 12.0
Średnia: 6.07
Mediana: 6.00
Liczba braków: 148

Statystyki kolumny: ekgday
Min: 1.0
Maks: 31.0
Średnia: 15.58
Mediana: 15.00
Liczba braków: 153

Statystyki kolumny: ekgyr
Min: 1981.0
Maks: 1989.0
